In [ ]:
## Eolicos xls2kmz
## Versión para executar na plataforma Colab: colab.research.google.com

## Script que procesa coordenadas en formato UTM e produce un ficheiro
## KMZ preparado para traballar con GoogleMaps e Google Earth

## INPUT: folla de cálculo .xls ou .xlsx
## 

## A folla de cálculo ten diferentes follas para diferentes elementos:
## aeroxeradores, poligonais, LAATs etc.
## campos: id/nome ; coord X ; coord Y   (coordenadas UTM)

## OUTPUT: ficheiro KMZ

In [ ]:
# CONFIGURACIÓN

# Os ficheiros de entrada deben estar nunha carpeta do Drive
carpeta = 'gdrive/MyDrive/Datos_proxectos_enerxeticos/input/'

# Nome do ficheiro:
nome_ficheiro = 'nome_ficheiro.xlsx'

# Indica o CRS, o sistema de coordenadas
crs_orixe = 'EPSG:25829' #Huso 29
#crs_orixe = 'EPSG:25829' #Huso 30
crs_destino = 'epsg:4326'

In [ ]:
# Montar drive para acceder ás follas de cálculo
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Instala e importa as librarías extra necesarias
import pandas as pd
import numpy as np
!pip install pyproj
from pyproj import CRS, Transformer
!pip install simplekml
import simplekml

In [ ]:
# Lectura do contido do ficheiro, que garda nun único DataFrame
ficheiro = carpeta+nome_ficheiro
df = pd.read_excel(ficheiro)

In [ ]:
# Lectura das diferentes follas de datos
# Crea un novo DataFrame para cada folla
df_aeroxeradores = pd.read_excel(ficheiro, sheet_name='Aeroxeradores')
df_poligonal = pd.read_excel(ficheiro, sheet_name='Poligonal')
df_LAAT = pd.read_excel(ficheiro, sheet_name='LAAT')
df_LAAT2 = pd.read_excel(ficheiro, sheet_name='LAAT_2')
df_meteotorres = pd.read_excel(ficheiro, sheet_name='TorresMeteo')
df_subestacion = pd.read_excel(ficheiro, sheet_name='PoligonalSubestacion')
df_seccionamento = pd.read_excel(ficheiro, sheet_name='CentroSeccionamento')
df_outros = pd.read_excel(ficheiro, sheet_name='OutrosMarcadores')

In [ ]:
# Crea un obxecto kml ao que se engadirán todos os puntos, polígonos etc
kml=simplekml.Kml()

In [ ]:
# Configuración da transformación de coordenadas
transformer = Transformer.from_crs(crs_orixe,crs_destino,always_xy=True)

In [ ]:
# Aeroxeneradores
if not df_aeroxeradores.empty:
  for index, row in df_aeroxeradores.iterrows():
    modelLink = simplekml.Link(href = "untitled.dae")
    coordsSet = transformer.transform(row['x'],row['y'])
    coords = simplekml.Location(longitude=coordsSet[0], latitude=coordsSet[1], altitude=0)
    model = kml.newmodel(name=row['id'],description='aeroxenerador',altitudemode='relativeToGround', location=coords, link=modelLink)
    model.style.iconstyle.icon.href = "molino.png" # icono para gmaps

In [ ]:
# LAAT
if not df_LAAT.empty:
  lista_coords = []
  for index, row in df_LAAT.iterrows():
    coords = transformer.transform(row['x'],row['y'])
    lista_coords.append(coords)

    punto = kml.newpoint(name=row['id'])
    punto.coords = [coords]
    punto.style.iconstyle.scale = 0.7
    punto.style.iconstyle.icon.href = "laat.png"

  linea = kml.newlinestring(name="LAAT", description="Linea Aerea Alta Tensión", coords=lista_coords)
  linea.style.linestyle.color = simplekml.Color.rgb(2,136,209)
  linea.style.linestyle.width= 5  # 5 pixels

In [ ]:
# LAAT_2
if not df_LAAT_2.empty:
  lista_coords = []
  for index, row in df_LAAT_2.iterrows():
    coords = transformer.transform(row['x'],row['y'])
    lista_coords.append(coords)

    punto = kml.newpoint(name=row['id'])
    punto.coords = [coords]
    punto.style.iconstyle.scale = 0.7
    punto.style.iconstyle.icon.href = "laat.png"

  linea = kml.newlinestring(name="LAAT", description="Linea Aerea Alta Tensión", coords=lista_coords)
  linea.style.linestyle.color = simplekml.Color.rgb(2,136,209)
  linea.style.linestyle.width= 5  # 5 pixels

In [ ]:
# Torres meteorolóxicas
if not df_meteotorres.empty:
  for index, row in df_meteotorres.iterrows():
    punto = kml.newpoint(name=row['id'], coords=[transformer.transform(row['x'],row['y'])])
    punto.style.iconstyle.scale = 0.8  # Icon thrice as big
    punto.style.iconstyle.icon.href = "torremet.png"

In [ ]:
# Poligonal
if not df_poligonal.empty:
  lista_coords = []
  for index, row in df_poligonal.iterrows():
    coords = transformer.transform(row['x'],row['y'])
    lista_coords.append(coords)
  lista_coords.append(lista_coords[0])

  poligono = kml.newpolygon(name="Poligonal")
  poligono.outerboundaryis = lista_coords
  poligono.style.linestyle.color = simplekml.Color.rgb(255,82,82)
  poligono.style.linestyle.width = 1
  poligono.style.polystyle.color = simplekml.Color.changealphaint(50, simplekml.Color.rgb(255,82,82))

In [ ]:
# Poligonal Subestación
if not df_subestacion.empty:
  lista_coords = []
  for index, row in df_subestacion.iterrows():
    coords = transformer.transform(row['x'],row['y'])
    lista_coords.append(coords)
  lista_coords.append(lista_coords[0])

  poligono = kml.newpolygon(name="Poligonal Subestación")
  poligono.outerboundaryis = lista_coords
  poligono.style.linestyle.color = simplekml.Color.rgb(255,234,0)
  poligono.style.linestyle.width = 1
  poligono.style.polystyle.color = simplekml.Color.changealphaint(50, simplekml.Color.rgb(255,234,0))

In [ ]:
# Centro de seccionamento
if not df_seccionamento.empty:
  for index, row in df_seccionamento.iterrows():
    punto = kml.newpoint(name=row['id'], coords=[transformer.transform(row['x'],row['y'])])
    punto.style.iconstyle.scale = 0.8  # Icon thrice as big
    punto.style.iconstyle.icon.href = "sub.png"    

In [ ]:
# Outros marcadores
if not df_outros.empty:
  for index, row in df_outros.iterrows():
    punto = kml.newpoint(name=row['id'], coords=[transformer.transform(row['x'],row['y'])])
    punto.style.iconstyle.scale = 0.8  # Icon thrice as big
    punto.style.iconstyle.icon.href = "sub.png"

In [ ]:
## Escritura dos ficheiros en disco
# Crea o ficheiro temporal doc.kml cos datos creados en apartados anteriores
# Crea un ficheiro kmz que contén o ficheiro doc.kml mais os ficheiros de imaxe

# Os ficheiros coas imaxes e obxectos 3D deben estar na mesma carpeta que a
# a folla de cálculo.

!pip install zipp
from zipfile import ZipFile
import os

os.chdir('/content/'+carpeta)

kml.save('doc.kml')

novo_ficheiro = nome_ficheiro[0:nome_ficheiro.rindex('.')]+".kmz"

zipObj = ZipFile(novo_ficheiro, 'w')
zipObj.write('doc.kml')

zipObj.write('molino.png')
zipObj.write('torremet.png')
zipObj.write('laat.png')
zipObj.write('untitled.dae')
zipObj.write('sub.png')

zipObj.close()
os.remove('doc.kml')